## convert using docker 
https://github.com/NVIDIA/TensorRT-LLM/tree/main/examples/bert

### tensorrt-llm version 0.12.0

1. docker run -it --rm --network host -v ./:/data --gpus "device=1" trtllm_tritonserver bash
2. pip install pycuda tensorflow h5py==3.10.0 'transformers<=4.42.4'
3. cd /data/third_party/TensorRT-LLM/examples/bert
4. code convert

NOTES:
1. comment these following line:
    - line 129 /usr/local/lib/python3.10/dist-packages/tensorrt_llm/profiler.py
    ```
    Comment
        if pynvml.__version__ < '11.5.0' or driver_version < '526':
            logger.warning(
                f'Found pynvml=={pynvml.__version__} and cuda driver version '
                f'{driver_version}. Please use pynvml>=11.5.0 and cuda '
                f'driver>=526 to get accurate memory usage.')
            # Support legacy pynvml. Note that an old API could return
            # wrong GPU memory usage.
            _device_get_memory_info_fn = pynvml.nvmlDeviceGetMemoryInfo
        else:
    ```
    => Only reserve after `else`, then fix index and save file


##### custom mask

In [ ]:
# error:
[02/02/2025-04:25:43] [TRT] [I] Global timing cache in use. Profiling results in this builder pass will be stored.
[02/02/2025-04:25:43] [TRT] [E] IBuilder::buildSerializedNetwork: Error Code 4: Internal Error (kOPT values for profile 0 violate shape constraints: BertRetriever/layers/0/attention/__add___L321/elementwise_binary_L2855/ELEMENTWISE_SUM_0: dimensions not compatible for elementwise. Broadcast has incompatible dimensions: 128 != 256 && 128 != 1 && 256 != 1.)
Traceback (most recent call last):
  File "/data/third_party/TensorRT-LLM/examples/bert/build_retrieve.py", line 273, in <module>
    engine = builder.build_engine(network, builder_config)
  File "/usr/local/lib/python3.10/dist-packages/tensorrt_llm/_common.py", line 204, in decorated
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tensorrt_llm/builder.py", line 411, in build_engine
    assert engine is not None, 'Engine building failed, please check the error log.'
AssertionError: Engine building failed, please check the error log.

##### fp32

In [ ]:
python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint"

python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint"

In [14]:
# add new file for customizing input 
python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint"

python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint"

SyntaxError: invalid syntax (206555486.py, line 3)

##### fp16

In [ ]:
python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/bert-retrieve-qry-base_float16_tllm_checkpoint"

python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint"

In [ ]:
python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint"

python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_tllm_checkpoint"


##### fp32 + bert_attetion_plugin float32

In [ ]:
python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_bert_atten_plugin" \
    --use_bert_attention_plugin float32

python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_bert_atten_plugin" \
    --use_bert_attention_plugin float32

In [ ]:
python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_bert_atten_plugin" \
    --use_bert_attention_plugin float32

python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_bert_atten_plugin" \
    --use_bert_attention_plugin float32

##### fp16 + bert_attetion_plugin float16

In [ ]:
python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_bert_atten_plugin" \
    --use_bert_attention_plugin float16

python3 build.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float32 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_bert_atten_plugin" \
    --use_bert_attention_plugin float16

In [ ]:
python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-qry-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_bert_atten_plugin" \
    --use_bert_attention_plugin float16

python3 build_retrieve.py \
    --model BertModel \
    --model_dir="/data/mbert-retrieve-ctx-base/" \
    --dtype=float16 --log_level=verbose \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_bert_atten_plugin" \
    --use_bert_attention_plugin float16

### tensorrt_llm v0.16.0

1. docker run -it --rm --network host -v ./:/data --gpus "device=1" trtllm_tritonserver bash
2. pip install pycuda tensorflow h5py==3.10.0 'transformers<=4.42.4'
3. cd /data/third_party/TensorRT-LLM/examples/bert
4. code convert

##### fp32

In [ ]:
# v0.16

python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/data/mbert-retrieve-qry-base/"   \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2"  \
    --dtype float32  \
    --tp_size 1

trtllm-build --checkpoint_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2"  \
    --output_dir="/data/outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2" \
    --remove_input_padding=enable \
    --bert_attention_plugin=float32 \
    --max_batch_size 8 \
    --max_input_len 512

python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/data/mbert-retrieve-ctx-base/"   \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2"  \
    --dtype float32  \
    --tp_size 1


trtllm-build --checkpoint_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2"  \
    --output_dir="/data/outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2" \
    --remove_input_padding=enable \
    --bert_attention_plugin=float32 \
    --max_batch_size 16 \
    --max_input_len 512


##### fp16

In [ ]:
# v0.16

python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/data/mbert-retrieve-qry-base/"   \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_tllm_checkpoint_2"  \
    --dtype float16  \
    --tp_size 2

trtllm-build --checkpoint_dir "/data/outputs/trtllm/mbert-retrieve-qry-base_float16_tllm_checkpoint_2"  \
    --output_dir="/data/outputs/trtllm/mbert-retrieve-qry-base_float16_tllm_checkpoint_2" \
    --remove_input_padding=enable \
    --bert_attention_plugin=float16 \
    --max_batch_size 10 \
    --max_input_len 512

python3 convert_checkpoint.py \
    --model BertModel  \
    --model_dir "/data/mbert-retrieve-ctx-base/"   \
    --output_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint_2"  \
    --dtype float16  \
    --tp_size 2


trtllm-build --checkpoint_dir "/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint_2"  \
    --output_dir="/data/outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint_2" \
    --remove_input_padding=enable \
    --bert_attention_plugin=float16 \
    --max_batch_size 8 \
    --max_input_len 512


## Evaluate

In [1]:
import os
os.environ['CUDA_DEVICE'] = '0'

### utils

In [2]:
# becnhmark run onnx model
import tensorrt as trt
import numpy as np
import os

import pycuda.driver as cuda
import pycuda.autoinit


from transformers import AutoTokenizer

class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()

class TrtModel:
    
    def __init__(self,engine_path,max_batch_size=1,dtype=np.float32):
        
        self.engine_path = engine_path
        self.dtype = dtype
        self.logger = trt.Logger(trt.Logger.WARNING)
        self.runtime = trt.Runtime(self.logger)
        self.engine = self.load_engine(self.runtime, self.engine_path)
        self.max_batch_size = max_batch_size
        # self.inputs, self.outputs, self.bindings = self.allocate_buffers()
        self.stream = cuda.Stream()
        self.context = self.engine.create_execution_context()

                
    @staticmethod
    def load_engine(trt_runtime, engine_path):
        trt.init_libnvinfer_plugins(None, "")             
        with open(engine_path, 'rb') as f:
            engine_data = f.read()
        engine = trt_runtime.deserialize_cuda_engine(engine_data)
        return engine
    
    def allocate_buffers(self, binding_shape, input_lengths):
        # Allocate host and device buffers
        inputs, outputs, bindings = [], [], []
        for binding in self.engine:
            
            if self.engine.get_tensor_mode(binding) == trt.TensorIOMode.INPUT and binding != "input_lengths":
                self.context.set_input_shape(binding, binding_shape)
            elif binding == "input_lengths": 
                self.context.set_input_shape(binding, input_lengths)

            # print("binding: ", binding)
            size = trt.volume(self.context.get_tensor_shape(binding))
            # print("size: ", size)
            # print("batch_size: ", self.context.get_tensor_shape(binding))
            dtype = trt.nptype(self.engine.get_tensor_dtype(binding))
            # print("dtype: ", dtype)

            host_mem = cuda.pagelocked_empty(size, dtype)
            device_mem = cuda.mem_alloc(host_mem.nbytes)
            bindings.append(int(device_mem))

            # if self.engine.binding_is_input(binding):
            if self.engine.get_tensor_mode(binding) == trt.TensorIOMode.INPUT:
                inputs.append(HostDeviceMem(host_mem, device_mem))
            else:
                outputs.append(HostDeviceMem(host_mem, device_mem))

        return inputs, outputs, bindings
        
            
    def __call__(self, inputs_id, attention_mask, token_type_ids, batch_size=2):

        
        x = np.array(inputs_id).astype(self.dtype)
        y = np.array(attention_mask).astype(self.dtype)
        z = np.array(token_type_ids).astype(self.dtype)
        input_lengths = np.array([batch_size]).astype(self.dtype)

        inputs, outputs, bindings = self.allocate_buffers(x.shape, input_lengths)
    
            
        # Transfer input data to the GPU.
        # print(x.shape)
        np.copyto(inputs[0].host,x.ravel())
        np.copyto(inputs[1].host,y.ravel())
        np.copyto(inputs[2].host,z.ravel())
        np.copyto(inputs[3].host, input_lengths.ravel())
        
        # after copy -> transfer to device, transer first will error duo to hold old value
        for inp in inputs:
            cuda.memcpy_htod_async(inp.device, inp.host, self.stream)

        # Run inference
        self.context.execute_v2(bindings=bindings)
        
        # Transfer prediction output from the GPU.
        for out in outputs:
            cuda.memcpy_dtoh_async(out.host, out.device, self.stream)
        
        # Synchronize the stream
        self.stream.synchronize()
        return [out.host.reshape(batch_size,-1) for out in outputs]
    

    

/home/tiennv/.conda/envs/trt-hung-10_7/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import time

def encode_trt(texts, model, tokenizer, batch_size):
    # check if tokenize length is min 128
    encoded_input = tokenizer(
        texts, 
        padding='max_length', 
        truncation=True,
        max_length=128,
        return_tensors='np'
    )

    start_time = time.time()
    embeddings = model(
        encoded_input['input_ids'],
        encoded_input['attention_mask'],
        encoded_input['token_type_ids'],
        batch_size
    )[0]
    end_time = time.time() - start_time

    # print(embeddings.reshape(batch_size, -1, 768))
    return embeddings.reshape(batch_size, -1, 768)[:, 0], end_time

In [4]:
from torch import nn
import torch

cross_entropy = nn.CrossEntropyLoss(reduction='mean')

def compute_loss(scores, target):
    return cross_entropy(scores, target)

def compute_similarity(q_reps, p_reps):
    if not isinstance(q_reps, torch.Tensor):
        q_reps = torch.tensor(q_reps)
    if not isinstance(p_reps, torch.Tensor):
        p_reps = torch.tensor(p_reps)
    return torch.matmul(q_reps, p_reps.transpose(0,1))

In [5]:
from tqdm import tqdm
import torch
import time
from typing import Callable
import inspect

def eval_accuracy_trt(
    data, 
    encode_fn = Callable, 
    num_passages=65, 
    model_qry=None, 
    model_ctx=None, 
    tokenizer_query=None,
    tokenizer_ctx=None, 
    device='cpu',
):

    assert model_ctx is not None, "model_ctx is required"
    assert model_qry is not None, "model_qry is required"
    assert tokenizer_ctx is not None, "tokenizer_ctx is required"
    assert tokenizer_query is not None, "tokenizer_query is required"
    assert 'query' in data.column_names, "data must have query column"
    assert 'positive' in data.column_names, "data must have positive column"
    assert 'negatives' in data.column_names, "data must have negatives column"
    # len of arguemtn of encode_fn must be 4
    # print(inspect.getargspec(encode_fn).args)
    assert len(inspect.getargspec(encode_fn).args) == 4, "encode_fn must have 4 arguments"

    accuracy = 0

    if device != "cpu":
        model_ctx = model_ctx.to(device)
        model_qry = model_qry.to(device)

    time_query_total = 0
    time_query_run = 0
    time_passage_total = 0
    time_passage_run = 0

    for i in tqdm(range(len(data))):

        start_time = time.time()
        #! CHANGE HERE
        query_batch = [data[i]['query']]
        query, time_query = encode_fn(query_batch, model_qry, tokenizer_query, len(query_batch))
        end_time = time.time() - start_time
        time_query_total += end_time
        time_query_run += time_query

        # concate 10 passages
        concate_passage = [data[i]['positive']] + data[i]['negatives'][:num_passages-1]
        start_time = time.time()
        #! CHANGE HERE
        encoded_passages, time_ctx = encode_fn(concate_passage, model_ctx, tokenizer_ctx, len(concate_passage))
        end_time = time.time() - start_time
        time_passage_total += end_time
        time_passage_run += time_ctx

        # accuracy
        scores = compute_similarity(query, encoded_passages)
        if scores.argmax(dim=1).detach().numpy() != 0:
            continue
        accuracy += 1

    return accuracy / len(data), time_query_run/ len(data), time_passage_run/ len(data), time_query_total/ len(data), time_passage_total/ len(data)

### prepare datasets

In [6]:
import datasets
from datasets import concatenate_datasets
en_eval = datasets.load_dataset('tiennv/mmarco-passage-vi', split='train[-500:]', cache_dir="../datahub/")
vi_eval = datasets.load_dataset('tiennv/mmarco-passage-vi', split='train[-500:]', cache_dir="../datahub/")

dataset_eval = concatenate_datasets([en_eval, vi_eval])
dataset_eval

Using the latest cached version of the dataset since tiennv/mmarco-passage-vi couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at ../datahub/tiennv___mmarco-passage-vi/default/0.0.0/5ee2171bc2bc0880d2f35c16063096ec1c4dc4da (last modified on Sun Jan 26 14:00:08 2025).
Using the latest cached version of the dataset since tiennv/mmarco-passage-vi couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at ../datahub/tiennv___mmarco-passage-vi/default/0.0.0/5ee2171bc2bc0880d2f35c16063096ec1c4dc4da (last modified on Sun Jan 26 14:00:08 2025).


Dataset({
    features: ['query_id', 'query', 'positive_id', 'positive', 'negatives'],
    num_rows: 1000
})

### run

In [18]:
import numpy as np

# trtversion 10.3.0 - trtllm 0.12.0.dev...
# trt_engine_qry_path = "../outputs/trtllm/mbert-retrieve-qry-base_float16_tllm_checkpoint/BertModel_float16_tp1_rank0.plan"
# trt_engine_ctx_path = "../outputs/trtllm/mbert-retrieve-ctx-base_float16_tllm_checkpoint/BertModel_float16_tp1_rank0.plan"

trt_engine_qry_path = "../outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint/BertModel_float32_tp1_rank0.plan"
trt_engine_ctx_path = "../outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint/BertModel_float32_tp1_rank0.plan"

model_query = TrtModel(trt_engine_qry_path, max_batch_size=1, dtype=np.int32)
model_ctx = TrtModel(trt_engine_ctx_path, max_batch_size=10, dtype=np.int32)
tokenizer_qry = AutoTokenizer.from_pretrained("../mbert-retrieve-qry-base")
tokenizer_ctx = AutoTokenizer.from_pretrained("../mbert-retrieve-ctx-base/")

[02/12/2025-10:15:15] [TRT] [E] IRuntime::deserializeCudaEngine: Error Code 1: Internal Error (Failed due to an old deserialization call on a newer plan file. This might happen when the plan file was built from an older TensorRT version. You can use `trtexec --getPlanVersionOnly` to check the version of TensorRT that was used to create the plan file.)


AttributeError: 'NoneType' object has no attribute 'create_execution_context'

In [8]:
# device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
accuracy, time_query_run, time_passage_run, time_query_total, time_passage_total = eval_accuracy_trt(
    dataset_eval, 
    encode_trt,
    num_passages=10, 
    model_ctx=model_ctx,
    model_qry=model_query, 
    tokenizer_ctx=tokenizer_ctx,
    tokenizer_query=tokenizer_qry,
    device="cpu"
)
print(f"Accuracy: {accuracy}")
print(f"Time Query Run: {time_query_run}")
print(f"Time Passage Run: {time_passage_run}")
print(f"Time Query Total: {time_query_total}")
print(f"Time Passage Total: {time_passage_total}")

/tmp/ipykernel_1457919/162317057.py:27: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  assert len(inspect.getargspec(encode_fn).args) == 4, "encode_fn must have 4 arguments"
  0%|          | 0/1000 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'set_input_shape'

### run with attention plugin
use code run of tensorrtllm

In [7]:
import argparse
import json
import os

# isort: off
import torch
import tensorrt as trt
# isort: on

import tensorrt_llm
from tensorrt_llm import logger
from tensorrt_llm.runtime import Session, TensorInfo

# from build import get_engine_name  # isort:skip
def get_engine_name(model, dtype, tp_size, rank):
    return '{}_{}_tp{}_rank{}.plan'.format(model, dtype, tp_size, rank)

def trt_dtype_to_torch(dtype):
    if dtype == trt.float16:
        return torch.float16
    elif dtype == trt.float32:
        return torch.float32
    elif dtype == trt.int32:
        return torch.int32
    else:
        raise TypeError("%s is not supported" % dtype)

2025-02-12 10:57:04.482372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739332624.495831 1784633 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739332624.499844 1784633 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 10:57:04.514305: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[TensorRT-LLM] TensorRT-LLM version: 0.16.0


In [8]:
qry_engine_dir = "../outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2/"
ctx_engine_dir = "../outputs/trtllm/mbert-retrieve-ctx-base_float32_tllm_checkpoint_2/"


In [9]:
import torch
import tensorrt_llm

tensorrt_llm.logger.set_level("info")

def get_model_config(config_path):
    
    config_path = os.path.join(qry_engine_dir, 'config.json')

    with open(config_path, 'r') as f:
        config = json.load(f)

    # V0.16.0 add builder 
    # assert config["plugin_config"]["remove_input_padding"] == False, \\
    # assert config['build_config']["plugin_config"]["remove_input_padding"] == False, \
    #     "Please refer to run_remove_input_padding.py for running BERT models with remove_input_padding enabled"
    return config

def get_session(config_path):
    config = get_model_config(config_path)
    # world_size = config['builder_config']['tensor_parallel']
    world_size = config['build_config']['auto_parallel_config']['world_size']
    assert world_size == tensorrt_llm.mpi_world_size(), \
        f'Engine world size ({world_size}) != Runtime world size ({tensorrt_llm.mpi_world_size()})'

    runtime_rank = tensorrt_llm.mpi_rank() if world_size > 1 else 0
    runtime_mapping = tensorrt_llm.Mapping(world_size,
                                            runtime_rank,
                                            tp_size=world_size)
    
    torch.cuda.set_device(runtime_rank % runtime_mapping.gpus_per_node)
    # dtype = config['builder_config']['precision']
    # model_name = config['builder_config']['name']
    # serialize_path = get_engine_name(
    #     model_name, 
    #     dtype, world_size, runtime_rank
    # )
    serialize_path = 'rank0.engine'
    serialize_path = os.path.join(qry_engine_dir, serialize_path)
    print(serialize_path)

    logger.info(f'Loading engine from {serialize_path}')
    with open(serialize_path, 'rb') as f:
        engine_buffer = f.read()

    logger.info(f'Creating session from engine')
    return Session.from_serialized_engine(engine_buffer)


In [12]:
import time

def encode_trtllm(texts, model, tokenizer, stream):
    # check if tokenize length is min 128
    encoded_input = tokenizer(
        texts, 
        padding='max_length', 
        truncation=True,
        max_length=512,
        return_tensors='pt', # need return pt here
        return_attention_mask=False,
    )

    output_info = model.infer_shapes([
        TensorInfo('input_ids', trt.DataType.INT32, encoded_input['input_ids'].numpy().ravel().shape), # flatten to set shape
        # TensorInfo('attention_mask', trt.DataType.INT32, encoded_input['attention_mask'].shape),
        TensorInfo('token_type_ids', trt.DataType.INT32, encoded_input['token_type_ids'].numpy().ravel().shape),
    ])
    print(output_info)
    outputs = {
        t.name: torch.empty(
            tuple((len(texts), t.shape[-1])),
            dtype=trt_dtype_to_torch(t.dtype),
            device='cuda'
        )
        for t in output_info
    }

    start_time = time.time()
    ok = model.run(encoded_input, outputs, stream)
    assert ok, "Runtime execution failed"
        
    end_time = time.time() - start_time

    torch.cuda.synchronize()
    embeddings = outputs['embeddings']

    # print(embeddings.reshape(batch_size, -1, 768))
    return embeddings.reshape(len(texts), -1, 768)[:, 0], end_time

In [13]:
accuracy, time_query_run, time_passage_run, time_query_total, time_passage_total = eval_accuracy_trt(
    dataset_eval, 
    encode_trtllm,
    num_passages=10, 
    model_ctx=get_session(ctx_engine_dir), 
    model_qry=get_session(qry_engine_dir), 
    tokenizer_ctx=AutoTokenizer.from_pretrained("../mbert-retrieve-ctx-base/"),
    tokenizer_query=AutoTokenizer.from_pretrained("../mbert-retrieve-qry-base/"),
    device="cpu"
)
print(f"Accuracy: {accuracy}")
print(f"Time Query Run: {time_query_run}")  
print(f"Time Passage Run: {time_passage_run}")
print(f"Time Query Total: {time_query_total}")
print(f"Time Passage Total: {time_passage_total}")

../outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2/rank0.engine
[02/12/2025-11:00:41] [TRT-LLM] [I] Loading engine from ../outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2/rank0.engine
[02/12/2025-11:00:41] [TRT-LLM] [I] Creating session from engine
[02/12/2025-11:00:41] [TRT] [I] Loaded engine size: 677 MiB
[02/12/2025-11:00:41] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[02/12/2025-11:00:41] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +77, now: CPU 0, GPU 2258 (MiB)
../outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2/rank0.engine
[02/12/2025-11:00:41] [TRT-LLM] [I] Loading engine from ../outputs/trtllm/mbert-retrieve-qry-base_float32_tllm_checkpoint_2/rank0.engine
[02/12/2025-11:00:41] [TRT-LLM] [I] Creating session from engine
[02/12/2025-11:00:41] [TRT] [I] Loaded engine size: 

/tmp/ipykernel_1784633/162317057.py:27: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  assert len(inspect.getargspec(encode_fn).args) == 4, "encode_fn must have 4 arguments"


[02/12/2025-11:00:42] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +76, now: CPU 0, GPU 3010 (MiB)


  0%|          | 0/1000 [00:00<?, ?it/s]

[TensorInfo(name='hidden_states', dtype=<DataType.FLOAT: 0>, shape=(-1, 768))]
[02/12/2025-11:00:42] [TRT] [E] IExecutionContext::enqueueV3: Error Code 3: API Usage Error (Parameter check failed, condition: mContext.profileObliviousBindings.at(profileObliviousIndex) != nullptr. Address is not set for input tensor position_ids. Call setInputTensorAddress or setTensorAddress before enqueue/execute.)


AssertionError: Runtime execution failed

## Appendix

combine calibration (API settings dfefault)

STATUS: 
- [Not supported model](https://github.com/NVIDIA/TensorRT-LLM/issues/1614#issuecomment-2122086630) 
- Config model type không có Bert - [line 110](https://github.com/NVIDIA/TensorRT-LLM/blob/main/tensorrt_llm/quantization/quantize_by_modelopt.py#L550)


### prepare

In [ ]:
import datasets
from datasets import concatenate_datasets 

number_samples = 250 
en = datasets.load_dataset('tiennv/mmarco-passage-vi', split=f'train[:{number_samples}]',
                          cache_dir="./datahub")
vi = datasets.load_dataset('tiennv/mmarco-passage-vi', split=f'train[:{number_samples}]', cache_dir="./datahub")

dataset_calib = concatenate_datasets([en, vi])
dataset_calib

### calib API

In [ ]:
from transformers import DataCollatorWithPadding, AutoTokenizer

query_tokenizer = AutoTokenizer.from_pretrained('mbert-retrieve-qry-base/')
ctx_tokenizer = AutoTokenizer.from_pretrained('mbert-retrieve-ctx-base/')

def query_collate_fn(examples):
    query = [example['query'] for example in examples]
    encoded_input = query_tokenizer(
        query, 
        padding='max_length', 
        truncation=True, 
        max_length=512, 
        return_tensors='pt'
    )
    return encoded_input


def ctx_collate_fn(examples):

    concate_passage = []
    for example in examples:
        concate_passage.extend(
            [example['positive']] + example['negatives'][:9]
        )

    # concate_passage = [examples['positive']] + examples['negatives'][:9]
    encoded_input = ctx_tokenizer(
        concate_passage, 
        padding='max_length', 
        truncation=True, 
        max_length=512, 
        return_tensors='pt',
        return_attention_mask=False
    )
    return encoded_input


In [ ]:
# Prepare the calibration set and define a forward loop
import torch

from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer

batch_size = 4
calib_batches = number_samples*2 // batch_size

num_workers = 4
device = "cuda:0" if torch.cuda.is_available() else "cpu" #! in docker with gpus specify, device_id is 0

calib_query_loader = torch.utils.data.DataLoader(
    dataset_calib, 
    batch_size=batch_size,
    collate_fn=query_collate_fn,
    num_workers=num_workers, 
    pin_memory=True
)

calib_ctx_loader = torch.utils.data.DataLoader(
    dataset_calib, 
    batch_size=batch_size,
    collate_fn=ctx_collate_fn,
    num_workers=num_workers, 
    pin_memory=True
)


query_model = AutoModel.from_pretrained('mbert-retrieve-qry-base/', return_dict=True)
ctx_model = AutoModel.from_pretrained('mbert-retrieve-ctx-base/', return_dict=True)

query_model.to(device)
ctx_model.to(device)
print("Initialize ...")

In [ ]:
from tqdm import tqdm
def calibrate_loop_query():
    for i, (encode_input) in tqdm(enumerate(calib_query_loader), total=calib_batches):
        for k, v in encode_input.items():
            encode_input[k] = v.to(device)
            # print(k, v.shape)
        query_model(**encode_input)
        if i >= calib_batches:
            break

def calibrate_loop_ctx():
    for i, (encode_input) in tqdm(enumerate(calib_ctx_loader), total=calib_batches):
        for k, v in encode_input.items():
            encode_input[k] = v.to(device)
            # print(k, v.shape)
        ctx_model(**encode_input)
        if i >= calib_batches:
            break

### quantize

In [ ]:
import modelopt.torch.quantization as atq
# Select the quantization config, for example, FP8
config = atq.FP8_DEFAULT_CFG
# PTQ with in-place replacement to quantized modules
with torch.no_grad():
    atq.quantize(query_model, config, forward_loop=calibrate_loop_query)

In [ ]:
from modelopt.torch.export import export_tensorrt_llm_checkpoint

decoder_type="bert"
dtype=torch.float32
export_dir="./mbert-retrieve-qry-base-quantize-trtllm-fp8"
# ["fp8", "int8_sq", "int4_awq", "w4a8_awq", "int8_wo", "int4_wo", "full_prec"]
qformat="fp8"
DEFAULT_MAX_SEQ_LEN=512
tp_size=1
pp_size=1
awq_block_size=128
# ["int8", "fp8", None]
kv_cache_dtype="int8"


with torch.inference_mode():
    export_tensorrt_llm_checkpoint(
        model=query_model,  # The quantized model.
        decoder_type="bert",
        # decoder_type,  # The type of the model as str, e.g gptj, llama or gptnext.
        dtype=dtype,  # The exported weights data type as torch.dtype.
        export_dir=export_dir,  # The directory where the exported files will be stored.
        inference_tensor_parallel=tp_size,  # The tensor parallelism size for inference.
        inference_pipeline_parallel=pp_size,  # The pipeline parallelism size for inference.
    )

In [ ]:
# Select the quantization config, for example, FP8
config = atq.FP8_DEFAULT_CFG
# PTQ with in-place replacement to quantized modules
with torch.no_grad():
    atq.quantize(ctx_model, config, forward_loop=calibrate_loop_ctx)

In [ ]:
from modelopt.torch.export import export_tensorrt_llm_checkpoint

decoder_type="bert"
dtype=torch.float32
export_dir="./mbert-retrieve-ctx-base-quantize-trtllm-fp8"
# ["fp8", "int8_sq", "int4_awq", "w4a8_awq", "int8_wo", "int4_wo", "full_prec"]
qformat="fp8"
DEFAULT_MAX_SEQ_LEN=512
tp_size=1
pp_size=1
awq_block_size=128
# ["int8", "fp8", None]
kv_cache_dtype="int8"

with torch.inference_mode():
    export_tensorrt_llm_checkpoint(
        model=ctx_model,  # The quantized model.
        decoder_type="bert",
        # decoder_type,  # The type of the model as str, e.g gptj, llama or gptnext.
        dtype=dtype,  # The exported weights data type as torch.dtype.
        export_dir=export_dir,  # The directory where the exported files will be stored.
        inference_tensor_parallel=tp_size,  # The tensor parallelism size for inference.
        inference_pipeline_parallel=pp_size,  # The pipeline parallelism size for inference.
    )